# Preprocessing
Processes claim and article data <br>
TODO: Fix read_json

In [7]:
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
# import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [8]:
SOURCE= "utils/"
DEST = "data/"
HOME = "./"

In [9]:
all_data = pd.read_json(DEST+"all_training_data.json")

ValueError: Unmatched ''"' when when decoding 'string'

In [5]:
def remove_between_square_brackets(text):
    """Remove all text between squared brakets"""
    return re.sub('\[[^]]*\]', '', text)


def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def tokenize(text):
    """Tokenized a text into words"""
    return nltk.word_tokenize(text)


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words


def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words


def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            try:
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            except:
                pass
        else:
            new_words.append(word)
    return new_words


def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems


def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas


def find_similar(tfidf_matrix, index, top_n=10):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]


def normalize(text, claim, claimant):
    text = remove_between_square_brackets(text)
    text = replace_contractions(text)
    claim = remove_between_square_brackets(claim)
    claim = replace_contractions(claim).replace(".", " ")
    sentences = str(claimant + " " + claim + ". " + text).split(".")
    
    for i, s in enumerate(sentences):
        words = tokenize(s)
        words = remove_non_ascii(words)
        words = to_lowercase(words)
        words = remove_punctuation(words)
        words = replace_numbers(words)
#         words = remove_stopwords(words)
        sentences[i] = " ".join(words)

    vect = TfidfVectorizer(stop_words="english")
    X = vect.fit_transform(sentences)
    top_ = find_similar(X, 0, top_n=5)
        
    important_sentences = sentences[0] + " "
    for a, (i,score) in enumerate(top_):
        important_sentences += (sentences[i]) + " "
    
    return tokenize(important_sentences)

In [6]:
def preprocess_all_data(data, norm=False, lem_stem=False):
    data["related_articles_with_metadata"] = ""
    
    if norm:
        data["normalized_articles"] = ""

    if lem_stem: 
        data["stemmed_articles"] = ""
        data["lemmatized_articles"] = ""
        data["stemmed_and_lemmatized_articles"] = ""
    
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        
        articles_with_metadata = row.claimant + " " + row.claim + ". " + ". ".join(row.related_articles)
        data.at[idx, "related_articles_with_metadata"] = articles_with_metadata
        
        if norm:
            s = ". ".join(row.related_articles)
            normalized_articles = normalize(s, row.claim)
            normalized_articles = normalized_articles[:500]   # keep only top 500 words (including the claim)
            data.at[idx, "normalized_articles"] = normalized_articles
            
        if lem_stem:
            stemmed_articles = stem_words(normalized_articles)
            lemmatized_articles = lemmatize_verbs(normalized_articles)
            stemmed_and_lemmatized_articles = lemmatize_verbs(stemmed_articles)
            data.at[idx, "stemmed_articles"] = stemmed_articles
            data.at[idx, "lemmatized_articles"] = lemmatized_articles
            data.at[idx, "stemmed_and_lemmatized_articles"] = stemmed_and_lemmatized_articles

    return data

In [7]:
all_data_preprocessed = preprocess_all_data(all_data.copy()[:])

In [12]:
all_data_preprocessed.to_json(DEST+"all_training_data_preprocessed_7.json")

In [85]:
all_data_preprocessed_test = pd.read_json(DEST+"all_training_data_preprocessed_7_v1.json")
all_data_preprocessed_test.claim

Exception ignored in: <bound method tqdm.__del__ of 100%|<bar/>| 17045/17045 [58:02<00:00,  4.89it/s]>
Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.5/site-packages/tqdm/std.py", line 1038, in __del__
    def __del__(self):
KeyboardInterrupt


KeyboardInterrupt: 

In [13]:
# all_data_preprocessed_tiny.related_articles[10]

In [2]:

a = np.zeros(7408)
b = np.zeros(6451) + 1
c = np.zeros(1696) + 2

y_true = np.concatenate([a,b,c])
y_preds = np.concatenate([a,b,np.zeros(1696)-1])

NameError: name 'np' is not defined

In [136]:
from sklearn.metrics import f1_score

def score1(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro') 

print(score1(y_true, y_preds))

0.6283529332497789
